<a href="https://colab.research.google.com/github/gyeongdong13/Competition/blob/main/%ED%86%B5%ED%95%A9%EB%B3%B8%EC%84%A0(ANN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/시니어/통합본선 최종 - 이상치 변환.csv', encoding='utf-8')

df.head()

,행정동,계총인구수,요보호 노인인구 비율,평균소득,총가구수,영역둘레(평균상권),영역면적(평균상권),의료기관수,1의료기관 당 요보호 노인인구 (종속변수),행정동 면적,...,보건소,보건지소,보건진료소,상급종합,의원,종합병원,치과병원,치과의원,한방병원,한의원
0,가음정동,38875.0,2.22,4727.67,7924,657.13,35157.54,37.0,0.06,2.89,...,0,0,0,0,15.0,0,0,12.0,0,9.0
1,가포동,2279.0,3.95,3001.30,15529,10229.23,2705034.00,1.0,3.95,4.47,...,0,0,0,0,0.0,0,0,0.0,0,0.0
2,경화동,10824.0,6.23,2241.70,3198,1313.01,153242.20,16.0,0.39,2.12,...,0,0,0,0,6.0,0,0,3.0,0,7.0
3,교방동,15947.0,7.73,2323.80,19918,1008.87,108515.20,10.0,0.77,2.73,...,0,0,0,0,3.0,0,0,4.0,0,3.0
4,구산면,4072.0,16.90,2139.00,14839,12028.37,4010639.00,3.0,5.63,43.64,...,0,1,1,0,1.0,0,0,0.0,0,0.0


In [ ]:
df = df.iloc[:,1:]
df.head()

,계총인구수,요보호 노인인구 비율,평균소득,총가구수,영역둘레(평균상권),영역면적(평균상권),의료기관수,1의료기관 당 요보호 노인인구 (종속변수),행정동 면적,상권 개수,...,보건소,보건지소,보건진료소,상급종합,의원,종합병원,치과병원,치과의원,한방병원,한의원
0,38875.0,2.22,4727.67,7924,657.13,35157.54,37.0,0.06,2.89,83,...,0,0,0,0,15.0,0,0,12.0,0,9.0
1,2279.0,3.95,3001.30,15529,10229.23,2705034.00,1.0,3.95,4.47,2,...,0,0,0,0,0.0,0,0,0.0,0,0.0
2,10824.0,6.23,2241.70,3198,1313.01,153242.20,16.0,0.39,2.12,20,...,0,0,0,0,6.0,0,0,3.0,0,7.0
3,15947.0,7.73,2323.80,19918,1008.87,108515.20,10.0,0.77,2.73,21,...,0,0,0,0,3.0,0,0,4.0,0,3.0
4,4072.0,16.90,2139.00,14839,12028.37,4010639.00,3.0,5.63,43.64,11,...,0,1,1,0,1.0,0,0,0.0,0,0.0


# ElasticNet 이후 ANN

In [ ]:
x = df.iloc[:, [1,5,13,14]]
y = df.iloc[:, 7]    # label(마지막 하나만 가져오기)
x.head(5), y.head(5)

(   요보호 노인인구 비율  영역면적(평균상권)  행정동별 병원간의 평균거리  병원에서 가장 가까운 버스정류장 평균 거리
 0         2.22    35157.54            0.56                     0.20
 1         3.95  2705034.00            0.00                     0.09
 2         6.23   153242.20            0.35                     0.09
 3         7.73   108515.20            0.56                     0.11
 4        16.90  4010639.00            2.47                     0.10,
 0    0.06
 1    3.95
 2    0.39
 3    0.77
 4    5.63
 Name: 1의료기관 당 요보호 노인인구 (종속변수), dtype: float64)

In [ ]:
from sklearn.preprocessing  import StandardScaler # 표준화

scaler = StandardScaler()
xs = scaler.fit_transform(x) # 각 독립변수의 평균과 표준편차 이용하여 단위 축소
x = pd.DataFrame(xs, columns=x.columns)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [ ]:
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((44, 4), (11, 4), (44,), (11,))

In [ ]:
from tensorflow.keras.layers import Dense      # 완전연결층 => 가장 기본적인 딥러닝 층
from tensorflow.keras.models import Sequential # Keras 패키지의 Model 생성 함수

model = Sequential()                                           # Sequential은 layer을 순차적으로 쌓는다
model.add(Dense(24,  activation='relu'))                       # Dense (Fully Connected Layer, 완전연결층) 레이어의 입력과 출력을 빠짐없이 모두 연결함. # Node (neuron) 수   # 활성화 함수(activation function) relu (rectified linear unit) 딥러닝에서 가장 많이 사용
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [ ]:
epochs = 30
batch_size = 8

history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_split=0.2,  verbose=1)

Epoch 1/30
5/5 [==============================] - 0s 26ms/step - loss: 0.6867 - mse: 0.6867 - val_loss: 5.3714 - val_mse: 5.3714
Epoch 2/30
5/5 [==============================] - 0s 14ms/step - loss: 0.6729 - mse: 0.6729 - val_loss: 5.2815 - val_mse: 5.2815
Epoch 3/30
5/5 [==============================] - 0s 14ms/step - loss: 0.6589 - mse: 0.6589 - val_loss: 5.2115 - val_mse: 5.2115
Epoch 4/30
5/5 [==============================] - 0s 10ms/step - loss: 0.6457 - mse: 0.6457 - val_loss: 5.1601 - val_mse: 5.1601
Epoch 5/30
5/5 [==============================] - 0s 11ms/step - loss: 0.6360 - mse: 0.6360 - val_loss: 5.0962 - val_mse: 5.0962
Epoch 6/30
5/5 [==============================] - 0s 11ms/step - loss: 0.6279 - mse: 0.6279 - val_loss: 5.0468 - val_mse: 5.0468
Epoch 7/30
5/5 [==============================] - 0s 10ms/step - loss: 0.6198 - mse: 0.6198 - val_loss: 5.0105 - val_mse: 5.0105
Epoch 8/30
5/5 [==============================] - 0s 12ms/step - loss: 0.6102 - mse: 0.6102 - val

In [ ]:
from sklearn.metrics import mean_squared_error

# 학습 성능 (예측력) 확인하기: R² 와 RMSE (test data)
y_pred = model.predict(test_x)
R2 = r2_score(test_y, y_pred)    #r2_score(실재값, 예측값)
RMSE = mean_squared_error(test_y, y_pred)**(1/2)


# 학습 성능 (예측력) 확인하기
print('test data 성과평가', R2, RMSE)

1/1 [==============================] - 0s 66ms/step
test data 성과평가 0.46409521241838403 1.5910994269585557
